In [1]:
# Adapted from: https://github.com/lazyprogrammer/machine_learning_examples/tree/master/rl
# Value iteration
import numpy as np
from gridWorldGame import standard_grid, negative_grid,print_values, print_policy

SMALL_ENOUGH = 1e-3
GAMMA = 0.9
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')
# this grid gives you a reward of -0.1
# to find a shorter path to the goal, use negative grid
grid = negative_grid()
print("rewards:")
print_values(grid.rewards, grid)

rewards:
---------------------------
-0.10|-0.10|-0.10| 1.00|
---------------------------
-0.10| 0.00|-0.10|-1.00|
---------------------------
-0.10|-0.10|-0.10|-0.10|


In [2]:
# state -> action
# choose an action and update randomly 
policy = {}
for s in grid.actions.keys():
  policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)

In [3]:
# initial policy
print("initial policy:")
print_policy(policy, grid)

initial policy:
---------------------------
  L  |  U  |  U  |     |
---------------------------
  D  |     |  R  |     |
---------------------------
  U  |  L  |  L  |  U  |


In [4]:
# initialize V(s) - value function
V = {}
states = grid.all_states()
for s in states:
  # V[s] = 0
  if s in grid.actions:
    V[s] = np.random.random()
  else:
    # terminal state
    V[s] = 0

# initial value for all states in grid
print(V)
print_values(V, grid)

{(0, 1): 0.9081669898685976, (1, 2): 0.7545508914625857, (2, 1): 0.12083392598409415, (0, 2): 0.8117018409876225, (2, 2): 0.23275195515807823, (1, 0): 0.22657215812083054, (1, 3): 0, (0, 0): 0.9807801087788558, (0, 3): 0, (2, 0): 0.3124503606393648, (2, 3): 0.551313376504095}
---------------------------
 0.98| 0.91| 0.81| 0.00|
---------------------------
 0.23| 0.00| 0.75| 0.00|
---------------------------
 0.31| 0.12| 0.23| 0.55|


In [5]:
# this section is different from policy iteration
# repeat until convergence
# V[s] = max[a]{ sum[s',r] { p(s',r|s,a)[r + gamma*V[s']] } }
iteration=0
while True:
  iteration+=1
  print("values %d: " % iteration)
  print_values(V, grid)
  print("policy %d: " % iteration)
  print_policy(policy, grid)
  
  biggest_change = 0
  for s in states:
    old_v = V[s]

    # V(s) only has value if it's not a terminal state
    if s in policy:
      new_v = float('-inf')
      for a in ALL_POSSIBLE_ACTIONS:
        grid.set_state(s)
        r = grid.move(a)
        v = r + GAMMA * V[grid.current_state()]
        if v > new_v:
          new_v = v
      V[s] = new_v
      biggest_change = max(biggest_change, np.abs(old_v - V[s]))

  if biggest_change < SMALL_ENOUGH:
    break

# find a policy that leads to optimal value function
for s in policy.keys():
  best_a = None
  best_value = float('-inf')
  # loop through all possible actions to find the best current action
  for a in ALL_POSSIBLE_ACTIONS:
    grid.set_state(s)
    r = grid.move(a)
    v = r + GAMMA * V[grid.current_state()]
    if v > best_value:
      best_value = v
      best_a = a
  policy[s] = best_a

# our goal here is to verify that we get the same answer as with policy iteration
print("values:")
print_values(V, grid)
print("policy:")
print_policy(policy, grid)

values 1: 
---------------------------
 0.98| 0.91| 0.81| 0.00|
---------------------------
 0.23| 0.00| 0.75| 0.00|
---------------------------
 0.31| 0.12| 0.23| 0.55|
policy 1: 
---------------------------
  L  |  U  |  U  |     |
---------------------------
  D  |     |  R  |     |
---------------------------
  U  |  L  |  L  |  U  |
values 2: 
---------------------------
 0.78| 0.78| 1.00| 0.00|
---------------------------
 0.78| 0.00| 0.63| 0.00|
---------------------------
 0.60| 0.18| 0.47| 0.40|
policy 2: 
---------------------------
  L  |  U  |  U  |     |
---------------------------
  D  |     |  R  |     |
---------------------------
  U  |  L  |  L  |  U  |
values 3: 
---------------------------
 0.62| 0.80| 1.00| 0.00|
---------------------------
 0.60| 0.00| 0.80| 0.00|
---------------------------
 0.44| 0.44| 0.62| 0.46|
policy 3: 
---------------------------
  L  |  U  |  U  |     |
---------------------------
  D  |     |  R  |     |
---------------------------
  U  